# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
# !pip install langchain
# !pip install -qU langchain-groq
import os

# Load a specific environment variable
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [4]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [5]:
#!pip install pandas

## LLMChain

In [8]:
!pip install langchain --upgrade
# from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain import tool, LLMChain

ImportError: cannot import name 'tool' from 'langchain' (C:\anaconda3\Lib\site-packages\langchain\__init__.py)

In [9]:
# llm = ChatOpenAI(temperature=0.9, model=llm_model)
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [10]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [11]:
chain = LLMChain(llm=llm, prompt=prompt)

C:\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [12]:
product = "Queen Size Sheet Set"
chain.run(product)

C:\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Here are some name suggestions for a company that makes Queen Size Sheet Set:\n\n1. **Royal Slumber**: This name plays off the idea of a queen\'s luxurious bedding and the idea of a comfortable sleep.\n2. **Queenly Sleep Co.**: This name incorporates the idea of a queen-sized bed and the company\'s focus on providing a comfortable sleeping experience.\n3. **DreamWeave**: This name suggests the idea of weaving together a comfortable and restful sleep, which is perfect for a company that makes sheet sets.\n4. **Kingdom Bedding**: This name evokes the idea of a regal and comfortable sleeping experience, with "kingdom" implying a sense of grandeur and quality.\n5. **SoftTouch Linens**: This name emphasizes the softness and comfort of the company\'s sheet sets, which is a key feature that customers look for in bedding.\n6. **Slumber Royale**: This name combines the ideas of sleep and royalty, suggesting a luxurious and comfortable sleeping experience.\n7. **Bedroom Bliss**: This name empha

## SimpleSequentialChain

In [13]:
from langchain.chains import SimpleSequentialChain

In [14]:
# llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [15]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [16]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [17]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Here are some name suggestions for a company that makes Queen-size sheet sets:

1. **Regal Linens**: This name plays off the idea of a queen-sized bed being regal and luxurious, and "linens" is a straightforward description of what the company produces.
2. **QueenCraft**: This name highlights the company's focus on queen-sized sheet sets while also emphasizing the craftsmanship that goes into making them.
3. **Sleepy Kingdom**: This name is whimsical and evocative, suggesting a cozy and comfortable sleeping experience. "Kingdom" implies a sense of grandeur and luxury.
4. **Bedding Bliss**: This name conveys a sense of happiness and satisfaction, implying that the company's sheet sets will help customers sleep better and feel more comfortable.
5. **Luxurium**: This name combines "luxury" and "ium" (a suffix suggesting high-quality materials), implying that the company's sheet sets are both luxurious and durable.
6. **DreamWeave**: This nam

'Here is a 20-word description for Regal Linens:\n\n"Regal Linens crafts luxurious Queen-size sheet sets with attention to detail, providing a regal sleeping experience for the discerning customer."'

## SequentialChain

In [18]:
from langchain.chains import SequentialChain

In [19]:
# llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [20]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [21]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [22]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [23]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [24]:
review = """Hertz, fácil de encontrar con excelente servicio. Alquilamos un Auto DS (Citroen), este es un vehículo excepcional y es un placer conducirlo. El mejor sistema de navegación que he usado y perfecto. El precio era un poco alto pero valió la pena.
Cliente feliz."""
overall_chain(review)

C:\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Hertz, fácil de encontrar con excelente servicio. Alquilamos un Auto DS (Citroen), este es un vehículo excepcional y es un placer conducirlo. El mejor sistema de navegación que he usado y perfecto. El precio era un poco alto pero valió la pena.\nCliente feliz.',
 'English_Review': 'Here is the translation:\n\n"Hertz, easy to find with excellent service. We rented a DS Auto (Citroen), this is an exceptional vehicle and it\'s a pleasure to drive it. The best navigation system I\'ve ever used and perfect. The price was a bit high, but it was worth it.\nHappy customer."',
 'summary': 'Here is a summary of the review in 1 sentence:\n\nThe reviewer had a positive experience with Hertz, praising the easy location, excellent service, and exceptional vehicle (a Citroen DS Auto) with a great navigation system, despite the higher price.',
 'followup_message': "Here's a follow-up response in Spanish:\n\nMe alegra leer que tu experiencia con Hertz fue tan positiva. Me parece interesante

## Router Chain

In [25]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [28]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [26]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
# llm = ChatOpenAI(temperature=0, model=llm_model)

In [29]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [30]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [31]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [32]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [33]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [34]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation in the context of thermodynamics?'}
> Finished chain.


"Black body radiation is a fundamental concept in thermodynamics that deals with the interaction between matter and electromagnetic radiation.\n\nIn simple terms, a black body is an idealized object that absorbs all the electromagnetic radiation that falls on it, without reflecting or transmitting any of it. In other words, it's a perfect absorber of radiation.\n\nWhen a black body is heated, it emits radiation across a wide range of frequencies, including visible light, infrared, and even ultraviolet radiation. This emitted radiation is known as black body radiation.\n\nThe key characteristic of black body radiation is that it follows a specific distribution, known as the Planck distribution, which was derived by Max Planck in the early 20th century. This distribution describes how the intensity of radiation emitted by a black body varies with frequency and temperature.\n\nFor example, when a black body is heated to a higher temperature, it emits more radiation in the visible spectrum

In [35]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'A classic!\n\nTo answer this question, I\'ll break it down into its component parts:\n\n1. What does "2" mean?\nIn mathematics, "2" is a number, specifically a positive integer. It represents a quantity that can be counted or measured.\n2. What does the "+" symbol mean?\nThe "+" symbol is an operator, specifically an addition operator. It indicates that we are combining two quantities, in this case, the two "2"s.\n3. What happens when we combine two "2"s?\nTo combine two "2"s using addition, we count the total number of units. In this case, we have two groups of 2 units each. When we combine these groups, we get a total of 4 units.\n\nNow, let\'s put it all together:\n\n2 + 2 = ?\n\nAnswer: 4!\n\nSo, the correct answer is 2 + 2 = 4.'

In [36]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
physics: {'input': 'Why does every cell in our body contain DNA, specifically?'}
> Finished chain.


'What a great question!\n\nThe answer lies in the fundamental concept of heredity and the mechanisms of biological inheritance. You see, DNA (Deoxyribonucleic acid) is the molecule that contains the genetic instructions used in the development and function of all living organisms. It\'s often referred to as the "blueprint" or "instructions" for life.\n\nIn the case of cells, DNA serves as the primary repository of genetic information, which is crucial for the cell\'s survival, growth, and reproduction. The DNA molecule is made up of two complementary strands that are twisted together to form a double helix structure. This structure is incredibly compact, allowing a vast amount of genetic information to be stored in a relatively small space.\n\nNow, every cell in our body contains DNA because it\'s essential for the cell\'s function. DNA provides the instructions for the cell to synthesize proteins, which are the building blocks of all biological structures and processes. It also contai

Reminder: Download your notebook to you local computer to save your work.

In [37]:
await llm.ainvoke(messages)

NameError: name 'messages' is not defined